## Download Datasets

### Download Cityscapes

In [ ]:
# The following code will only execute
# successfully when compression is complete

import kagglehub

# Download latest version
path = kagglehub.dataset_download("alessiogio/cityscapes-correctlabels")

print("Path to dataset files:", path)

### Download Tiny Imagenet

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("akash2sharma/tiny-imagenet")

print("Path to dataset files:", path)

### Download fishyscapes- lost and found

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alessiogio/fs-l-and-f")

print("Path to dataset files:", path)

### Download SMIYC Road Anomaly 21

In [ ]:
# The following code will only execute
# successfully when compression is complete

import kagglehub

# Download latest version
path = kagglehub.dataset_download("alessiogio/smiyc-road-anomaly-21")

print("Path to dataset files:", path)

### Download SMIYC Road Obstacles 21

In [ ]:
# The following code will only execute
# successfully when compression is complete

import kagglehub

# Download latest version
path = kagglehub.dataset_download("alessiogio/smiyc-road-obstacles-21")

print("Path to dataset files:", path)

## Git Clone

In [ ]:
!git clone https://github.com/Nome-Provvisorio/AnomalySegmentation.git

In [ ]:
!git pull origin main

In [ ]:
!pip install visdom
!pip install cv2
#!pip install torch_xla

#!pip install cityscapesscripts
!pip install ood_metrics

## Training

### Run Pretrain

In [ ]:
%cd "/kaggle/working"
!python3 "AnomalySegmentation/imagenet/main.py" "/kaggle/input/tiny-imagenet/tiny-imagenet-200"

### Run training

In [ ]:
%cd "/kaggle/working/AnomalySegmentation/train"
!python3 "main.py" --savedir erfnet_training1 --datadir "/kaggle/input/cityscapes-correctlabels/Cityscape" --num-epochs 10 --batch-size 1 --decoder --pretrainedEncoder "/kaggle/working/AnomalySegmentation/train/model_best.pth.tar"

## Convertitore Labels

In [ ]:

# python imports
from __future__ import print_function, absolute_import, division
import os, glob, sys

# cityscapes imports
from cityscapesscripts.helpers.csHelpers import printError
from cityscapesscripts.preparation.json2labelImg import json2labelImg

# The main method
def main():
    # Where to look for Cityscapes

    cityscapesPath = "/kaggle/input/cityscapes/Cityscape"
    # how to search for all ground truth
    searchFine   = os.path.join( cityscapesPath , "gtFine"   , "*" , "*" , "*_gt*_polygons.json" )
    searchCoarse = os.path.join( cityscapesPath , "gtCoarse" , "*" , "*" , "*_gt*_polygons.json" )

    # search files
    filesFine = glob.glob( searchFine )
    filesFine.sort()
    filesCoarse = glob.glob( searchCoarse )
    filesCoarse.sort()

    # concatenate fine and coarse
    files = filesFine + filesCoarse
    # files = filesFine # use this line if fine is enough for now.

    # quit if we did not find anything
    if not files:
        printError( "Did not find any files. Please consult the README." )

    # a bit verbose
    print("Processing {} annotation files".format(len(files)))

    # iterate through files
    progress = 0
    print("Progress: {:>3} %".format( progress * 100 / len(files) ), end=' ')
    for f in files:
        # create the output filename
        dst = f.replace( "_polygons.json" , "_labelTrainIds.png" )

        # do the conversion
        try:
            json2labelImg( f , dst , "trainIds" )
        except:
            print("Failed to convert: {}".format(f))
            raise

        # status
        progress += 1
        print("\rProgress: {:>3} %".format( progress * 100 / len(files) ), end=' ')
        sys.stdout.flush()


# call the main
if __name__ == "__main__":
    main()

## Evaluation

### Generates color images for visualization

In [ ]:
!python3 "/kaggle/working/AnomalySegmentation/eval/eval_cityscapes_color.py" --datadir "/kaggle/input/cityscapes-correctlabels/Cityscape" --subset val --loadDir "/kaggle/working/" --loadWeights "AnomalySegmentation/save/erfnet_training1/model_best.pth" 

### eval_iou

In [ ]:
!python  "eval_iou.py" --datadir "datasets\cityscapes" --loadDir "AnomalySegmentation" --loadWeights "/trained_models/model_best_log_norm.pth" --subset val

### eval anomaly

In [ ]:
 !python "evalAnomaly.py" --input "Validation_Dataset\fs_static\images" --loadDir "/AnomalySegmentation" --loadWeights "/trained_models/model_best_log_norm.pth" --metric maxlogit